In [18]:
from google.colab import files
uploaded = files.upload()

Saving DS4200 Data (Responses) - Form Responses 1.csv to DS4200 Data (Responses) - Form Responses 1 (1).csv


In [19]:
import pandas as pd
import numpy as np

# Loads CSV
csv = "DS4200 Data (Responses) - Form Responses 1.csv"
df = pd.read_csv(csv, skiprows=1)

# Renames columns
df.columns = [
    "Timestamp", "Rental Type", "Roommates", "Bedrooms", "Bathrooms",
    "Furnished", "Amenities", "Total Rent", "Utilities Included",
    "Utility Cost", "Neighborhood", "Proximity to Transit", "Personal Income"
]

# Cleans whitespace
df.columns = df.columns.str.strip()
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)


# Converts money ranges or "Less than" to an average
def parse_money_range(value):
    if pd.isna(value) or "Not applicable" in value:
        return np.nan
    value = value.replace("$", "").replace(",", "").replace(" ", "")
    if "less" in value.lower():
        return 2000
    if "–" in value or "-" in value:
        delimiter = "–" if "–" in value else "-"
        try:
            low, high = value.split(delimiter)
            return (int(low) + int(high)) / 2
        except:
            return np.nan
    try:
        return int(value)
    except:
        return np.nan

# Extracts people from roommate text
def extract_people(text):
    if pd.isna(text):
        return np.nan
    import re
    match = re.search(r"(\d+)", text)
    if match:
        if "other" in text.lower():
            return int(match.group(1)) + 1
        return int(match.group(1))
    return np.nan

# Extracts all amenity types from the column
def extract_amenities(amenities):
    all_amenities = set()
    for entry in amenities.dropna():
        for item in entry.split(","):
            all_amenities.add(item.strip())
    return list(all_amenities)


# Applies parse money range function to the Total Rent, Utility Cost, and Personal Income columns
df["Total Rent Avg"] = df["Total Rent"].apply(parse_money_range)
df["Utility Cost Avg"] = df["Utility Cost"].apply(parse_money_range)
df["Personal Income Avg"] = df["Personal Income"].apply(parse_money_range)

# Drops rows where Total Rent Avg is 0 or missing
df = df[df["Total Rent Avg"].notna() & (df["Total Rent Avg"] != 0)]

# Extracts number of people
df["People in Unit"] = df["Roommates"].apply(extract_people)

# Converts Bedrooms and Bathrooms values to numeric
df["Bedrooms"] = pd.to_numeric(df["Bedrooms"], errors="coerce")
df["Bathrooms"] = pd.to_numeric(df["Bathrooms"], errors="coerce")


# Standardizes Furnished values
df["Furnished"] = df["Furnished"].str.lower().replace({
    "furnished": "Furnished",
    "unfurnished": "Unfurnished",
    "partially furnished": "Partially"
})

# Simplifies Proximity values
df["Proximity to Transit"] = df["Proximity to Transit"].str.extract(
    r"(Very close|Close|Far|Not accessible)", expand=False)

# Encode amenities
unique_amenities = extract_amenities(df["Amenities"])
for amenity in unique_amenities:
    df[amenity] = df["Amenities"].apply(lambda x: 1 if isinstance(x, str) and amenity in x else 0)

# Drops unnecessary columns
df_cleaned = df.drop(columns=[
    "Timestamp", "Roommates", "Total Rent", "Utility Cost", "Personal Income", "Amenities",
    "none of these", "Water"
])

# Adds a "No Additional Amenities" column for comparison
amenity_cols = [col for col in unique_amenities if col not in ["Water", "none of these"]]
df_cleaned["No Additional Amenities"] = df_cleaned[amenity_cols].sum(axis=1).apply(lambda x: 1 if x == 0 else 0)

# Creates a new column for Total Rent including utilities
def compute_rent_with_utilities(row):
    if row["Utilities Included"] in ["No, no utilities included", "Yes, some utilities included"]:
        return row["Total Rent Avg"] + row["Utility Cost Avg"] if pd.notna(row["Utility Cost Avg"]) else row["Total Rent Avg"]
    else:
        return row["Total Rent Avg"]

df_cleaned["Total Rent w/ Utilities"] = df_cleaned.apply(compute_rent_with_utilities, axis=1)


# To see first couple of rows
print(df_cleaned.head())

# Exports the cleaned DataFrame to a CSV file to use for visualizations
output_path = "cleaned_rental_data.csv"
df_cleaned.to_csv(output_path, index=False)


# Downloads the CSV file
from google.colab import files
files.download(output_path)

                                         Rental Type  Bedrooms  Bathrooms  \
0                                          Apartment       2.0        1.0   
1                                          Apartment       3.0        1.0   
2                                          Apartment       3.0        1.0   
3                                          Apartment       4.0        1.0   
4  Luxury apartments (include gym, in-unit laundr...       2.0        1.0   

     Furnished            Utilities Included        Neighborhood  \
0    Furnished     No, no utilities included  The Fenway/Kenmore   
1    Partially  Yes, some utilities included             Roxbury   
2  Unfurnished  Yes, some utilities included             Roxbury   
3    Partially     No, no utilities included             Roxbury   
4  Unfurnished     No, no utilities included        Mission Hill   

  Proximity to Transit  Total Rent Avg  Utility Cost Avg  Personal Income Avg  \
0           Very close          3500.0         

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>